# Mastering DNA/Protein Sequence Analysis in R: A Step-by-Step Tutorial for Beginners

Find more tutorials about Computer Vision, Microscopy, Biology and Data Science [here](https://medium.com/@microbioscopicdata)

![SegmentLocal](Animated_logo_smaller_faster.gif "segment")

<div style="text-align: justify"> Sequences are the fundamentals tools for biologists and bioinformaticians playing a crucial role in various domains of research. Sequence analysis has made significant contributions to biological studies and remains a fundamental task with broad applications in genetics, genomics, text mining, and time series analysis 1. Whether you are a biologist, a data scientist, or simply someone curious about exploring patterns and structures in sequences, this comprehensive tutorial aims to provide you with a solid foundation in DNA/protein sequence analysis using R. We will discuss how to read and store DNA/protein sequences, manipulate and transform sequences, find motifs, and calculate sequence statistics.</div>

## Install necessary packages

In [1]:
# Install the seqinr package if not already installed, uncomment the line below
# install.packages("seqinr") 

# Install the APE package if not already installed, uncomment the line below
# install.packages("ape")

# Install the Biostrings package using Bioconductor if not already installed, uncomment the line below
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install("Biostrings")

## Import packages

In [63]:
library(seqinr);
library(ape);
library(rentrez);
library(Biostrings);

## Creating, downloading, reading and saving sequences in R
<div style="text-align: justify">Biological sequences are most often nucleotide or protein sequences usually represented as simple strings of characters. In R, DNA or protein sequences are typically represented as character vectors, which are simple strings of characters.</div>

In [104]:
# DNA sequence as character vector
dna_seq <- "ATGCTGACTGCTAGCGATCGTAGC"
print(dna_seq)

[1] "ATGCTGACTGCTAGCGATCGTAGC"


<div style="text-align: justify">In R we can retrieve sequences data from NCBI (National Center for Biotechnology Information) using the rentrez package. In this tutorial, we will demonstrate how to download (and save in FASTA format, see below) gene sequences associated with plasma-membrane-associated protein complexes in the model filamentous fungus <i>Aspergillus nidulans</i>, such as <i>pilA</i> and <i>pilB</i> 3,4. 
The <i>pilA</i> (and <i>pilB</i>) genes encode for BAR-domain-containing proteins that localize to eisosomes, which are intriguing cellular structures with multifaceted functions. Eisosomes are involved in crucial cellular processes, including plasma membrane organization, cell wall synthesis and morphogenesis, sphingolipid homeostasis, and stress response 5.</div>


In [106]:
# Fetch the sequence from NCBI - pilA
pilA <- rentrez::entrez_fetch(db = "nucleotide", 
                          id = "XM_657729.2",
                          rettype = "fasta")
# Save the sequence pilA in a FASTA file
write(pilA, file="pilA.fasta")

# Fetch the sequence from NCBI - pilB
pilB <- rentrez::entrez_fetch(db = "nucleotide", 
                          id = "XM_050611370.1",
                          rettype = "fasta")
# Save the sequence pilB in a FASTA file
write(pilB, file="pilB.fasta")

<div style="text-align: justify">Sequences are commonly saved and shared in the FASTA format, which is a simple and widely used format for storing biological sequences such as DNA or protein sequences2. In FASTA files each record (FASTA files can store one or multiple sequences/records) begin with a description line starting with a greater-than sign > character, followed on the next line by the sequences.</div>

`>XM_657729.2 Aspergillus nidulans FGSC A4 lipid-binding protein pilA (ANIA_05217), mRNA
TTTCGAACAGAAATCTGCTGTCCATCCACCAGTCAAGACGCGAATTGACTGTCGCGGCCCCAGTCGTCCC`


In [116]:
# Specify the file path
fasta_file <- "pilA.fasta"

# Read the FASTA file using read.fasta()
pilA <- read.fasta(file = fasta_file)


## Convert and clean FASTA sequences

To manipulate sequences and address various biological questions, it's essential to convert FASTA sequences into a suitable data structure in R 6.  . During the conversion process, there are several elements we need to remove or modify:  

•	Remove the identifier line, which typically starts with the ">" symbol and contains information about the sequence, such as the accession number and description.  
•	Eliminate all line breaks ("\n") that appear in the file, as they are not required for further analysis.  


•	(optional) Organize each nucleotide of the sequence into its individual position within a vector, allowing for efficient data manipulation. 

By performing these steps, we can obtain a clean and structured representation of the sequence data that is ready for further analysis and exploration in R.

### Remove the identifier line (metadata header). 

To remove  the identifier line we are going to use regular expressions. Regular expressions allow you to define patterns to find and replace specific characters or sequences.We are going to use gsub() function ito remove the pattern ^>.*?\n that matches the identifier line starting with >, followed by any characters .*?, and the newline character \n. 

^>: The ^ symbol asserts the beginning of a line, and > matches the greater-than symbol at the start of the metadata line. So, ^> matches any line that starts with >, which is typically the marker for the metadata lines in a FASTA file.  

.*?: The . matches any character except a newline. The * quantifier means zero or more occurrences of the previous character or group. The ? makes the * quantifier lazy, which means it will match as few characters as possible.   
This combination .*? matches any character lazily until the first occurrence of the following pattern.  

\n: The \n represents the newline character. It indicates the end of a line.
Therefore, the pattern "^>.*?\n" matches the metadata line from the beginning of the line until the first newline character.

In [122]:
# Fetch the sequence from NCBI - pilA
pilA <- rentrez::entrez_fetch(db = "nucleotide", 
                          id = "XM_657729.2",
                          rettype = "fasta")
pilA_cleaned <- gsub("^>.*?\n", "", replacement = "",x = pilA)
# Print the original sequence downloaded from NCBI (trim the sequence length to 120 characters for display)
strtrim(pilA,120)
# Print the cleaned sequence (trim the sequence length to 120 characters for display)
strtrim(pilA_cleaned,120)

[1] ">XM_657729.2 Aspergillus nidulans FGSC A4 lipid-binding protein pilA (ANIA_05217), mRNA\nTTTCGAACAGAAATCTGCTGTCCATCCACCAGT"

[1] "TTTCGAACAGAAATCTGCTGTCCATCCACCAGTCAAGACGCGAATTGACTGTCGCGGCCCCAGTCGTCCC\nAGATTGCTGGTCCTTATTTGAGAGTCACTAATACTGCTTGTGAGCTCGTG"

### Eliminate all line breaks ("\n")

<div style="text-align: justify">To remove unwanted characters, including the newline character (\n), you can use regular expressions with the gsub() function in R. In this case, we want to remove the \n characters from the sequence. Since the backslash () is a special character in regular expressions, we need to escape it by using two backslashes (\).</div>

In [125]:
# Remove all line breaks ("\n") using gsub function
pilA_vector <- gsub(pattern = "\\n", replacement = "",x = pilA_cleaned)

# Print the sequences (trim the sequence length to 120 characters for display)
strtrim(pilA_cleaned,75)
strtrim(pilA_vector,75)

[1] "TTTCGAACAGAAATCTGCTGTCCATCCACCAGTCAAGACGCGAATTGACTGTCGCGGCCCCAGTCGTCCC\nAGATT"

[1] "TTTCGAACAGAAATCTGCTGTCCATCCACCAGTCAAGACGCGAATTGACTGTCGCGGCCCCAGTCGTCCCAGATT"

### Organize each nucleotide of the sequence into its individual position within a vector

At last we need to split a continuous string of letters into a vector where each letter is separated, you can utilize the `str_split()` function from the stringr package

In [127]:
# Split the cleaned sequence into individual letters
pilA_vector_split <- stringr::str_split( pilA_vector ,pattern = "" ,simplify = FALSE)[[1]] # The [[1]] at the end retrieves the first element of the resulting list, which contains the split sequence as a character vector.
pilA_vector_split


[1] "T" "T" "T" "C" "G" "A" "A" "C" "A" "G" "A" "A" "A" "T" "C" "T" "G" "C"
  [19] "T" "G" "T" "C" "C" "A" "T" "C" "C" "A" "C" "C" "A" "G" "T" "C" "A" "A"
  [37] "G" "A" "C" "G" "C" "G" "A" "A" "T" "T" "G" "A" "C" "T" "G" "T" "C" "G"
  [55] "C" "G" "G" "C" "C" "C" "C" "A" "G" "T" "C" "G" "T" "C" "C" "C" "A" "G"
  [73] "A" "T" "T" "G" "C" "T" "G" "G" "T" "C" "C" "T" "T" "A" "T" "T" "T" "G"
  [91] "A" "G" "A" "G" "T" "C" "A" "C" "T" "A" "A" "T" "A" "C" "T" "G" "C" "T"
 [109] "T" "G" "T" "G" "A" "G" "C" "T" "C" "G" "T" "G" "A" "C" "T" "T" "C" "C"
 [127] "A" "T" "T" "T" "C" "C" "G" "G" "T" "T" "A" "G" "C" "G" "T" "C" "C" "A"
 [145] "A" "C" "C" "G" "A" "G" "T" "C" "T" "C" "A" "G" "A" "C" "G" "T" "C" "A"
 [163] "C" "C" "G" "A" "C" "C" "A" "T" "T" "T" "C" "C" "T" "C" "C" "A" "C" "T"
 [181] "C" "G" "A" "T" "C" "G" "G" "C" "T" "C" "T" "C" "T" "G" "C" "T" "G" "A"
 [199] "A" "C" "T" "G" "G" "A" "T" "T" "C" "T" "C" "C" "G" "G" "G" "G" "A" "A"
 [217] "A" "G" "A" "T" "C" "T" "C" "C" "T" "A" "G" "G" "T" "C" "A" "T" "C" "A"
 [235] "A" "C" "T" "T" "T" "A" "T" "T" "G" "C" "A" "T" "T" "C" "T" "C" "C" "T"
 [253] "C" "T" "C" "G" "T" "C" "G" "A" "G" "C" "T" "A" "C" "C" "G" "A" "T" "T"
 [271] "G" "C" "G" "G" "T" "T" "C" "T" "C" "C" "G" "G" "C" "C" "G" "T" "T" "C"
 [289] "C" "G" "C" "C" "G" "T" "T" "T" "C" "C" "G" "T" "G" "T" "G" "A" "C" "C"
 [307] "T" "C" "C" "T" "T" "A" "T" "A" "C" "C" "A" "C" "A" "C" "C" "A" "C" "G"
 [325] "T" "C" "A" "C" "T" "C" "G" "T" "A" "C" "G" "C" "A" "A" "A" "G" "C" "G"
 [343] "G" "C" "C" "T" "G" "A" "G" "G" "A" "A" "C" "T" "C" "A" "T" "T" "C" "C"
 [361] "G" "T" "C" "C" "A" "A" "T" "C" "C" "G" "G" "C" "T" "C" "A" "A" "T" "T"
 [379] "A" "G" "T" "A" "G" "A" "A" "T" "C" "T" "T" "C" "T" "T" "C" "T" "C" "C"
 [397] "T" "T" "C" "T" "C" "G" "T" "C" "G" "C" "C" "C" "A" "C" "A" "A" "T" "C"
 [415] "T" "C" "T" "G" "T" "T" "C" "T" "T" "T" "G" "C" "C" "G" "A" "T" "C" "T"
 [433] "C" "G" "T" "A" "C" "G" "C" "A" "G" "A" "A" "T" "A" "T" "C" "G" "G" "C"
 [451] "T" "G" "G" "T" "C" "T" "C" "T" "T" "C" "A" "C" "A" "A" "T" "G" "C" "A"
 [469] "T" "C" "G" "T" "A" "C" "A" "T" "A" "C" "T" "C" "T" "A" "T" "G" "C" "G"
 [487] "C" "C" "A" "G" "T" "C" "G" "C" "G" "G" "C" "T" "G" "C" "C" "G" "A" "C"
 [505] "C" "G" "C" "C" "T" "C" "G" "C" "A" "G" "A" "T" "C" "G" "A" "G" "A" "A"
 [523] "C" "C" "C" "C" "C" "C" "A" "C" "C" "A" "C" "C" "A" "C" "T" "C" "T" "C"
 [541] "C" "T" "C" "G" "A" "C" "C" "A" "A" "G" "A" "C" "G" "A" "A" "C" "A" "G"
 [559] "G" "T" "G" "G" "A" "T" "G" "G" "G" "G" "A" "A" "A" "G" "G" "T" "G" "G"
 [577] "T" "C" "T" "T" "G" "G" "G" "C" "A" "T" "G" "C" "C" "T" "T" "C" "C" "G"
 [595] "C" "A" "A" "A" "A" "A" "T" "G" "C" "T" "G" "C" "C" "G" "G" "T" "G" "C"
 [613] "T" "T" "T" "T" "G" "G" "A" "C" "C" "T" "G" "A" "C" "C" "T" "T" "G" "C"
 [631] "G" "C" "G" "C" "A" "A" "G" "C" "T" "C" "T" "C" "C" "C" "A" "G" "C" "T"
 [649] "G" "G" "T" "A" "A" "A" "G" "A" "T" "G" "G" "A" "A" "A" "A" "G" "A" "A"
 [667] "C" "G" "T" "C" "A" "T" "G" "C" "G" "C" "A" "G" "C" "A" "T" "G" "G" "A"
 [685] "A" "T" "T" "G" "G" "T" "C" "T" "C" "T" "C" "G" "G" "G" "A" "G" "C" "G"
 [703] "T" "A" "T" "G" "G" "A" "G" "A" "C" "T" "G" "C" "G" "C" "A" "A" "C" "A"
 [721] "A" "C" "T" "G" "T" "C" "C" "A" "T" "C" "T" "G" "G" "G" "G" "T" "G" "A"
 [739] "G" "A" "A" "C" "T" "G" "T" "G" "A" "C" "G" "A" "A" "G" "A" "C" "G" "T"
 [757] "G" "T" "C" "T" "G" "A" "T" "A" "T" "C" "A" "C" "G" "G" "A" "T" "A" "A"
 [775] "A" "C" "T" "G" "G" "G" "T" "G" "T" "C" "T" "T" "G" "C" "T" "C" "T" "A"
 [793] "T" "G" "A" "A" "G" "T" "G" "G" "G" "A" "G" "A" "G" "C" "T" "G" "G" "A"
 [811] "G" "G" "A" "C" "A" "T" "G" "T" "A" "C" "G" "T" "C" "G" "A" "C" "C" "G"
 [829] "C" "T" "A" "T" "G" "A" "C" "C" "A" "G" "T" "A" "C" "C" "G" "T" "G" "T"
 [847] "C" "A" "C" "C" "A" "T" "C" "A" "A" "G" "A" "G" "T" "A" "T" "C" "C" "G"
 [865] "C" "A" "A" "T" "A" "T" "T" "G" "A" "G" "G" "C" "G" "T" "C" "C" "G" "T"
 [883] "C" "C" "A" "G" "C" "C" "A" "A" "G" "C" "C" "G" "T" "G" "A" "C" "C" "G"
 [901] "C" "A" "A" "G" "C" "A" "G" "A" "A" "G" "A" "T

## Length and Base composition of a DNA sequence  


<div style="text-align: justify">Once you have retrieved a DNA sequence, we can obtain some simple statistics to describe that sequence, such as the sequence’s total length in nucleotides. In the above example, we retrieved the pilA sequence, and stored it in the vector variable pilA_vector_split. To obtain the length of the genome sequence, we would use the `length()` function.</div>

In [128]:
# Return the length of the pilA_vector_split vector
length (pilA_vector_split)

[1] 1741

<div style="text-align: justify">Every analysis of a DNA sequence often involves counting the occurrences of the four nucleotides: "A," "C," "G," and "T." This can be accomplished using the table() function. For instance, to determine the counts of A, C, G, and T in the pilA sequence (stored in the vector variable pilA_vector_split  as per the previous commands), you can use the following code:</div>

In [130]:
# Display  counts of each nucleotide in the sequence
table(pilA_vector_split)

pilA_vector_split
  A   C   G   T 
403 502 426 410 

## GC Content of DNA 

<div style="text-align: justify">One of the fundamental properties of a genome sequence is its GC content, which represents the percentage of Gs and Cs in the sequence. To calculate the GC content, we need to determine the number of Gs and Cs in the sequence and divide it by the total length of the genome.</div>

<div style="text-align: justify">For example, let's consider the pilA sequence. Using the table() function as mentioned earlier, we found that it contains 403 As, 502 Cs, 436 Gs, and 410 Ts. To calculate the GC content, you can use the following formula:  </div>

GC content = (number of Gs + number of Cs) / (genome length) * 100


In [132]:
# Manual calculation of GC content m
GC_content = ((502+436)/1741)*100
GC_content

[1] 53.87708

Alternatively, we can use the GC() function in the SeqinR package, which simplifies the calculation by directly providing the fraction of Gs and Cs in the sequence.

In [134]:
# Use of the function from the SeqinR package to calculate the GC content of a DNA sequence
seqinr::GC(pilA_vector_split) *100

[1] 53.3027

## Finding patterns/motifs 
<div style="text-align: justify">In the field of biology, finding patterns is a common task that involves identifying specific sequences or motifs within biological data such as DNA, RNA, or protein sequences. These patterns can provide valuable insights into the structure, function, and evolution of biological molecules.</div>

In [138]:
# Find the positions of a given motif within a sequence
motif <- "TAA"
words.pos(motif, pilA_vector, ignore.case = FALSE,perl = TRUE) #pilA_vector is a single line character vector 
# Print their frequency 
print(length(words.pos(motif, pilA_vector, ignore.case = FALSE,perl = TRUE)))

[1]   99  651  772 1321 1330 1505 1564 1591 1687 1710

[1] 10


<div style="text-align: justify">In conclusion, sequences play a critical role in biological research, serving as fundamental tools for analyzing genetic information. Through sequence analysis, researchers can gain valuable insights into various aspects of genetics and genomics. In this tutorial, we have covered essential topics related to sequence analysis using R. We have learned how to read and store sequences, manipulate and transform them, find motifs, and calculate sequence statistics.

## References  


1.	Ortutay, C. & Ortutay, Z. Molecular Data Analysis Using R. (Wiley-Blackwell, 2017).  

2.	Vangelatos, I. et al. Eisosome Organization in the Filamentous AscomyceteAspergillus nidulans. Eukaryot. Cell 9, 1441–1454 (2010).  

3.	Athanasopoulos, A., Gournas, C., Amillis, S. & Sophianopoulou, V. Characterization of AnNce102 and its role in eisosome stability and sphingolipid biosynthesis. Sci. Rep. 5, (2015).  

4.	Athanasopoulos, A., André, B., Sophianopoulou, V. & Gournas, C. Fungal plasma membrane domains. FEMS Microbiol. Rev. fuz022 (2019) doi:10.1093/femsre/fuz022.  

5.	Brouwer, A. C., with contributions by Nathan L. Chapter 21 Downloading DNA sequences as FASTA files in R | A Little Book of R for Bioinformatics 2.0.  

6.	Brouwer, A. C., with contributions by Nathan L. A Little Book of R for Bioinformatics 2.0.  


